In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pandas as pd
import numpy as np
import qgrid
from IPython.display import display, Markdown

In [13]:
# Reading the Excel files for the year 2023
df_PAQ_IMSS2023 = pd.read_excel(r'.\PAQs\IMSS2023.xlsx', usecols='A:J')
df_PAQ_INSABI2023 = pd.read_excel(r'.\PAQs\INSABI2023.xlsx', usecols='A:L')
df_PAQ_DESCENTRALIZADOS2023 = pd.read_excel(r'.\PAQs\DESCENTRALIZADOS2023.xlsx', usecols='A:L')

# Direcciones
path_IMSS2024 = r'C:\Users\armjorge\Dropbox\FACT 2024\Generacion facturas IMSS 2024.xlsx'
path_INSABI2024 = r'C:\Users\armjorge\Dropbox\FACT 2024\Consola IMSSB.xlsx'
path__DESCENTRALIZADOS2024 = r'C:\Users\armjorge\Dropbox\FACT 2024\Consola CCINSAHE.xlsx'
dir_razones_excluir = r'C:\Users\armjorge\Dropbox\3. Armando Cuaxospa\Reportes GPT\Integrado\RazonesNOCCINSHAE.xlsx'
dir_pagos_km = r'C:\Users\armjorge\Dropbox\3. Armando Cuaxospa\Reportes GPT\Dataframes\2023-2024pagos.xlsx'

# Carga
df_PAQ_IMSS2024 = pd.read_excel(path_IMSS2024, sheet_name='Reporte Paq', usecols='A:J')
df_PAQ_INSABI2024 = pd.read_excel(path_INSABI2024, sheet_name='Reporte PAQ', usecols='A:L')
df_PAQ_DESCENTRALIZADOS2024 = pd.read_excel(path__DESCENTRALIZADOS2024, sheet_name='Reporte Paq', usecols='A:L')
df_razones_no_CCINSHAE = pd.read_excel(dir_razones_excluir)
df_razones_no_CCINSHAE_lista = df_razones_no_CCINSHAE.iloc[:, 0].tolist()  # Adjust the index if necessary
df_pagos  = pd.read_excel(dir_pagos_km)
# Filtra
df_PAQ_IMSS2024 = df_PAQ_IMSS2024[df_PAQ_IMSS2024['Folio'].notna() & df_PAQ_IMSS2024['Folio'].str.strip().astype(bool)]
df_PAQ_INSABI2024 = df_PAQ_INSABI2024[df_PAQ_INSABI2024['UUID Descripción'] != 'Cancelado']
df_pagos_grouped = df_pagos.groupby('Folio').agg({
    'Fecha Pago': 'max',  # Get the most recent (newest) date
    'Importe pago': 'sum'  # Sum of 'Importe pago'
}).reset_index()


# Combining (concatenating) the DataFrames from 2023 and 2024
df_PAQ_IMSS_20232024 = pd.concat([df_PAQ_IMSS2023, df_PAQ_IMSS2024])
df_PAQ_INSABI_20232024 = pd.concat([df_PAQ_INSABI2023, df_PAQ_INSABI2024])
df_PAQ_DESCENTRALIZADOS_20232024 = pd.concat([df_PAQ_DESCENTRALIZADOS2023, df_PAQ_DESCENTRALIZADOS2024])

df_PAQ_IMSS_20232024.reset_index(drop=True, inplace=True)
df_PAQ_INSABI_20232024.reset_index(drop=True, inplace=True)

print(df_PAQ_DESCENTRALIZADOS_20232024.columns)
qgrid.show_grid(df_PAQ_DESCENTRALIZADOS_20232024, show_toolbar=True)


Index(['Fecha', 'Serie', 'Folio', 'Razón Social', 'Total', 'UUID',
       'UUID Descripción', 'Referencia', 'INSABI', 'Método de pago clave',
       'Uso CFDI', 'Forma de pago clave'],
      dtype='object')


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [10]:
# OPERACIONES CCINSHAES
df_PAQ_DESCENTRALIZADOS_20232024.reset_index(drop=True, inplace=True)
df_PAQ_DESCENTRALIZADOS_20232024 = df_PAQ_DESCENTRALIZADOS_20232024.dropna(subset=['Referencia'])
df_PAQ_DESCENTRALIZADOS_20232024 = df_PAQ_DESCENTRALIZADOS_20232024[df_PAQ_DESCENTRALIZADOS_20232024['Referencia'].str.startswith(('SO23', 'SO24'))]
df_PAQ_DESCENTRALIZADOS_20232024['Factura'] = df_PAQ_DESCENTRALIZADOS_20232024['Serie'] + '-' + df_PAQ_DESCENTRALIZADOS_20232024['Folio'].astype(str)
df_PAQ_DESCENTRALIZADOS_20232024['Factura'] = df_PAQ_DESCENTRALIZADOS_20232024['Factura'].str.replace('.0', '')

df_PAQ_DESCENTRALIZADOS_20232024 = df_PAQ_DESCENTRALIZADOS_20232024[df_PAQ_DESCENTRALIZADOS_20232024['UUID Descripción'] == 'Vigente']
df_PAQ_DESCENTRALIZADOS_20232024 = df_PAQ_DESCENTRALIZADOS_20232024[~df_PAQ_DESCENTRALIZADOS_20232024['Razón Social'].isin(df_razones_no_CCINSHAE_lista)]
df_PAQ_DESCENTRALIZADOS_20232024['Ciclo fiscal'] = np.where(
    df_PAQ_DESCENTRALIZADOS_20232024['Referencia'].str.startswith('SO23'), '2023',
    np.where(df_PAQ_DESCENTRALIZADOS_20232024['Referencia'].str.startswith('SO24'), '2024', 'Unknown')
)
print(df_PAQ_DESCENTRALIZADOS_20232024.columns)
qgrid.show_grid(df_PAQ_DESCENTRALIZADOS_20232024, show_toolbar=True)


Index(['Fecha', 'Serie', 'Folio', 'Razón Social', 'Total', 'UUID',
       'UUID Descripción', 'Referencia', 'INSABI', 'Método de pago clave',
       'Uso CFDI', 'Forma de pago clave', 'Factura', 'Ciclo fiscal'],
      dtype='object')


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [11]:


df_PAQ_DESCENTRALIZADOS2324_grouped = df_PAQ_DESCENTRALIZADOS_20232024.groupby('Factura').agg({
    'Total': 'sum',  # Sum the 'Total' for each group
    'Fecha': 'first',  # Keep the first 'Fecha' from each group
    'Serie': 'first',  # Keep the first 'Serie' from each group
    'Folio': 'first',  # Keep the first 'Folio' from each group
    'Razón Social': 'first',  # Keep the first 'Razón Social' from each group
    'UUID': 'first',  # Keep the first 'UUID' from each group
    'UUID Descripción': 'first',  # Keep the first 'UUID Descripción' from each group
    'Referencia': 'first',  # Keep the first 'Referencia' from each group
    'INSABI': 'first',  # Keep the first 'INSABI' from each group
    'Método de pago clave': 'first',  # Keep the first 'Método de pago clave' from each group
    'Uso CFDI': 'first',  # Keep the first 'Uso CFDI' from each group
    'Forma de pago clave': 'first',  # Keep the first 'Forma de pago clave' from each group
    'Ciclo fiscal': 'first'
})


print(df_PAQ_DESCENTRALIZADOS2324_grouped.columns)
qgrid.show_grid(df_PAQ_DESCENTRALIZADOS2324_grouped, show_toolbar=True)

Index(['Total', 'Fecha', 'Serie', 'Folio', 'Razón Social', 'UUID',
       'UUID Descripción', 'Referencia', 'INSABI', 'Método de pago clave',
       'Uso CFDI', 'Forma de pago clave', 'Ciclo fiscal'],
      dtype='object')


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:

fecha_pago_map = df_pagos_grouped.set_index('Folio')['Fecha Pago']
importe_pago_map = df_pagos_grouped.set_index('Folio')['Importe pago']

# Map 'Fecha Pago' and 'Importe pago' to df_PAQ_DESCENTRALIZADOS_20232024 based on 'Factura'
df_PAQ_DESCENTRALIZADOS2324_grouped['Fecha Pago'] = df_PAQ_DESCENTRALIZADOS2324_grouped['Factura'].map(fecha_pago_map)
df_PAQ_DESCENTRALIZADOS2324_grouped['Importe pago'] = df_PAQ_DESCENTRALIZADOS2324_grouped['Factura'].map(importe_pago_map)
df_PAQ_DESCENTRALIZADOS2324_grouped['Fecha Pago'] = pd.to_datetime(df_PAQ_DESCENTRALIZADOS2324_grouped['Fecha Pago']).dt.strftime('%d/%m/%Y')


In [48]:
# Setup Google Sheets API
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('key.json', scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1KN4XwXQlZ5jhyErxdpA_R6kNk2tKwwFeDnCDuHCr0fo/edit#gid=2033397596')

# Read the worksheets into DataFrames
def worksheet_to_df(worksheet_name, columns_range):
    worksheet = spreadsheet.worksheet(worksheet_name)
    data = worksheet.get(columns_range)
    return pd.DataFrame(data[1:], columns=data[0])
df_Zoho = worksheet_to_df('ZOHO', 'A:U')


df_Zoho['SalesOrder Number'] ['Institución Homologada', 'Contrato', 'Orden','lugarEntrega']

Index(['Fecha', 'Serie', 'Folio', 'Razón Social', 'Total', 'UUID',
       'UUID Descripción', 'Referencia', 'INSABI', 'Método de pago clave',
       'Uso CFDI', 'Forma de pago clave'],
      dtype='object')


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [45]:
print(df_PAQ_DESCENTRALIZADOS2324_grouped.columns)
qgrid.show_grid(df_PAQ_DESCENTRALIZADOS2324_grouped, show_toolbar=True)

Index(['Factura', 'Total', 'Fecha', 'Serie', 'Folio', 'Razón Social', 'UUID',
       'UUID Descripción', 'Referencia', 'INSABI', 'Método de pago clave',
       'Uso CFDI', 'Forma de pago clave', 'Ciclo fiscal', 'Fecha Pago',
       'Importe pago'],
      dtype='object')


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [49]:
print(df_Zoho.columns)
qgrid.show_grid(df_Zoho, show_toolbar=True)

Index(['Institución Homologada', 'Contrato', 'Orden', 'Clave',
       'fechaExpedicion', 'lugarEntrega', 'descripciónEntrega',
       'direccionEntrega', 'fechaEntrega', 'cantidadSolicitada', 'Precio',
       'SalesOrder Number', 'Status', 'Importe',
       'Fecha de entrega de la orden', 'Factura', 'Fecha de pago',
       'Ciclo fiscal', 'Estatus Factura', 'Fecha de Ingreso', 'Institución'],
      dtype='object')


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [4]:
print(df_PAQ_DESCENTRALIZADOS2324_grouped.columns)
qgrid.show_grid(df_PAQ_DESCENTRALIZADOS2324_grouped, show_toolbar=True)

Index(['Factura', 'Total', 'Fecha', 'Serie', 'Folio', 'Razón Social', 'UUID',
       'UUID Descripción', 'Referencia', 'INSABI', 'Método de pago clave',
       'Uso CFDI', 'Forma de pago clave', 'Ciclo fiscal', 'Fecha Pago',
       'Importe pago'],
      dtype='object')


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…